# MVD 4. cvičení

## 1. část - Načtení dat

Po rozbalení archive.zip uvidíte articles csv soubor. Tento soubor pochází z [Kaggle datasetů](https://www.kaggle.com/hsankesara/medium-articles) a obsahuje malé množství Medium článků k tématům ML, AI a data science. K úloze dnešního cvičení bude stačit využítí dat s názvy a obsahy článků (title a text).


### Příprava dat

Pro přípravu dat se použivá různá sekvence kroků. Je doporučeno na následující kroky vytvořit samostatnou funkci, aby bylo možné zpracovat i vyhledávaný výraz při testování. Dnešní cvičení by mělo obsahovat následující kroky:

1. Převést všechen text na lower case
2. Odstranění interpunkce a všech speciálních znaků (apostrof, ...)
3. Aplikace lemmatizátoru

Pozn.: Jedná se pouze o jednoduchý preprocessing, v praxi je často potřeba použití více kroků. Tato aplikace by měla například problém s čísly (desetinná čísla, čísla vyhledávaná slovně). 

Pro lemmatizaci použijte knihovnu spaCy.

In [ ]:
# Instalace spaCy z Jupyter Notebooku
import sys
!{sys.executable} -m pip install spacy

# Stažení modelu pro angličtinu
!{sys.executable} -m spacy download en

In [2]:
from collections import defaultdict, Counter
import csv
import math
import re
import numpy as np
import pandas as pd
import spacy
lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner']) # NLTK
# Lemmatizace textu př.:  
# " ".join([token.lemma_ for token in lemmatizer(text)])

### Funkce pro preprocessing textu

In [3]:
def to_lowercase(text: str) -> str:
    return text.lower()

# Simple test
print(to_lowercase("AbCdEfG HiG K L M. N10 T... xy{Z}"))

abcdefg hig k l m. n10 t... xy{z}


In [23]:
def remove_spec_chars(text: str) -> str:
    text = re.sub(r' +', ' ', text)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

# Simple test
print(remove_spec_chars("h@ello, m$y name   1is optimus p-r.i+m!e'"))

hello my name 1is optimus prime


In [24]:
def apply_lemmatizer(text: str, lemmatizer) -> list[str]:
    return " ".join([token.lemma_ for token in lemmatizer(text)])

print(apply_lemmatizer("I wanted to be loved by yours heart", lemmatizer))

I want to be love by yours heart


In [25]:
def preprocess(text: str) -> list[str]:
    text = to_lowercase(text)
    text = remove_spec_chars(text)
    text = apply_lemmatizer(text, lemmatizer=lemmatizer)
    return text

print(preprocess('Ab-!@#123 asdasd asd'))

ab123 asdasd asd


### Načítání članků z .csv souboru

In [26]:
csv_filename = "articles.csv"
titles = []
texts = []

with open(csv_filename, 'r') as file:
    articles = csv.reader(file)
    print(next(articles.__iter__()), end="\n\n")  # Printing the columns names
    
    for line in articles:
        titles.append(line[-2])
        texts.append(line[-1])

print(titles[0], end="\n\n")
print(texts[0])
print(f"Titles number: {len(titles)}, Texts number: {len(texts)}")

['author', 'claps', 'reading_time', 'link', 'title', 'text']

Chatbots were the next big thing: what happened? – The Startup – Medium

Oh, how the headlines blared:
Chatbots were The Next Big Thing.
Our hopes were sky high. Bright-eyed and bushy-tailed, the industry was ripe for a new era of innovation: it was time to start socializing with machines.
And why wouldn’t they be? All the road signs pointed towards insane success.
At the Mobile World Congress 2017, chatbots were the main headliners. The conference organizers cited an ‘overwhelming acceptance at the event of the inevitable shift of focus for brands and corporates to chatbots’.
In fact, the only significant question around chatbots was who would monopolize the field, not whether chatbots would take off in the first place:
One year on, we have an answer to that question.
No.
Because there isn’t even an ecosystem for a platform to dominate.
Chatbots weren’t the first technological development to be talked up in grandiose terms 

## 2. část - Vytvoření invertovaného indexu

Před další prací s textem je potřeba vytvořit invertovaný index, který poté usnadní práci. Invertovaný index bude slovník, kde klíčem bude slovo a hodnotou bude list s id dokumentů (index), které dané slovo obsahují.

Pozn.: Je potřeba vytvořit dva invertované indexy - jeden pro title a druhý pro text.

In [27]:
def create_inverted_index(doc_str_list: list[str]) -> defaultdict[list]:
    """
    Creates a dict, where:
        key: str -  word from text;
        value: list[int] - list of documents ids, where this word was located.
    Args:
        doc_str_list: list[str] - list of string documents texts
    Return:
        inv_idx_dict: defaultdict[list] - inverted indices dictionary, where there are all words from from all documents
    """
    inv_idx_dict = defaultdict(list)

    for doc_id, text in enumerate(doc_str_list):

        # Per word: add doc id to list of word ids if it isn't in list already
        for doc_w in text.split():
            if doc_id not in inv_idx_dict.get(doc_w, []):
                inv_idx_dict[doc_w].append(doc_id)

    return inv_idx_dict

In [28]:
# preprocess titles and texts of each document
prep_titles = [preprocess(title) for title in titles]
prep_texts = [preprocess(text) for text in texts]

titles_inv_idx = create_inverted_index(prep_titles)
texts_inv_idx = create_inverted_index(prep_texts)

In [ ]:
print(titles_inv_idx.get('machine', []))
print(texts_inv_idx.get('neural', []))
print(prep_titles[2])
print(prep_texts[2])

## 3. část - Implementace TF-IDF

Připravení funkce pro výpočet TF-IDF po příchodu dotazu. Funkce *tf_idf* by měla pracovat s dotazem, jedním invertovaným indexem a s danými dokumenty. Vrátit by měla list obsahující skóre pro každý dokument.

<br>
<center>
$
score(q,d) = TF\_IDF(q,d) = \sum\limits_{w \in q \cap d} c(w, q) c(w, d) log(\frac{M+1}{df(w)})
$
</center>

$q$ ... dotaz<br>
$d$ ... dokument<br>
$c(w, q)$ ... kolikrát je slovo *w* v dotazu *q*<br>
$M$ ... celkový počet dokumentů<br>
$df(w)$ ... počet dokumentů, ve kterých se nachází slovo *w*

In [67]:
def tf_idf(query: str, inverted_index: defaultdict[list], docs: list[str]) -> list: 
    M = len(docs)
    
    # Count the occurrences of each term in the query
    query_term_counts = defaultdict(int)
    for term in preprocess(query).split():
        query_term_counts[term] += 1
    # print(query_term_counts)
    
    # Initialize scores for each document
    scores = [0.0] * M
    
    # Calculate TF-IDF scores
    for term, term_count in query_term_counts.items():
        # print(f"1. Curr term: {term} ({term_count})")
        # Check if the term is in the inverted index
        if term in inverted_index:
            # Get the list of documents containing the term
            doc_id_list = inverted_index[term]
            # print(f"2. Doc ID list that cointain '{term}': \n{doc_id_list}")
            # Document frequency
            df_w = len(doc_id_list)
            # print(f"3. DF_w = {df_w}")
            # print(f"{term} - {doc_id_list}")
            
            # Calculate the contribution to the score for each document
            for doc_id in doc_id_list:
                # Count occurrences of the term in the document
                doc_term_count = docs[doc_id].split().count(term)
                # print(f"4. doc_id: {doc_id}, count in doc with curr ID: {doc_term_count}. Term: {term}")
                # TF-IDF score for the term in the document
                score_contribution = term_count * doc_term_count * math.log((M + 1) / df_w)
                # print(f"5. Score: {score_contribution}")
                # Update the score for the document
                # print(f"6. Curr score for this doc: {scores[doc_id]}")
                scores[doc_id] += score_contribution
                # print(f"  After: {scores[doc_id]}")

                # input(">>")
                # print("=" * 50)

    return scores


## 4. část - Použití a testování TF-IDF

Nyní lze získat skóre pro titulky nebo text. Následujícím krokem je sjednocení výsledného skóre pro ohodnocení celého dokumentu. V případě dvou hodnot si vystačíme s parametrem $\alpha$, který nám určuje jakou váhu má titulek a jakou samotný text dokumentu. <br>

<center>
$
score(q,d) = \alpha \; TF\_IDF\_title(q,d) + (1-\alpha) \; TF\_IDF\_text(q,d)
$
</center>

Při nastavení parametru $\alpha$ na hodnotu 0.7 a vyhledávání dotazu "coursera vs udacity machine learning" by výsledky měly vypadat následovně:

![output](sample_output.png)

In [68]:
alpha = 0.7
query = 'coursera vs udacity machine learning'
titles_scores = np.array(tf_idf(query, titles_inv_idx, prep_titles))
# print(len(titles_scores))
texts_scores = np.array(tf_idf(query, texts_inv_idx, prep_texts))

print(f"Title score for 276: {titles_scores[276]}")
print(f"Text score for 276: {texts_scores[276]}")

total_scores = alpha * titles_scores + (1 - alpha) * texts_scores

data = {
        'Title': prep_titles,
        'Text': prep_texts,
        'Score': total_scores
    }
df = pd.DataFrame(data)
df_sorted = df.sort_values(by='Score', ascending=False)
df_sorted
# print(df["Title"][276])
# print(df["Text"][276])
# print(df["Score"][276])

Title score for 276: 18.488162537198818
Text score for 276: 85.70941791888


,Title,Text,Score
276,coursera vs udacity for machine learn hacker...,2018 be an exciting time for student of machin...,38.654539
143,every single machine learning course on the in...,a year and a half ago I drop out of one of the...,19.560162
99,every single machine learning course on the in...,a year and a half ago I drop out of one of the...,19.560162
67,every single machine learning course on the in...,a year and a half ago I drop out of one of the...,19.560162
19,every single machine learning course on the in...,a year and a half ago I drop out of one of the...,19.560162
...,...,...,...
267,what be the good intelligent chatbot or ai cha...,how do we define the intelligence of a chatbot...,0.000000
33,multiindex locality sensitive hash for fun and...,one way that we deal with this volume of datum...,0.000000
286,multistream rnn concat rnn internal conv rnn l...,for the last two week I have be die to impleme...,0.000000
31,classify website with neural network knowled...,at datafiniti we have a strong need for conver...,0.000000
